In [ ]:
try:
    import cirq
    from iqm.cirq_iqm import Adonis, Apollo, circuit_from_qasm
    from iqm.cirq_iqm.optimizers import simplify_circuit
except ImportError:
    print('Installing missing dependencies...')
    !pip install --quiet cirq cirq_iqm
    from iqm.cirq_iqm import Adonis, Apollo, circuit_from_qasm
    from iqm.cirq_iqm.optimizers import simplify_circuit
    print('Installation ready')

# The Adonis architecture

Construct an `IQMDevice` instance representing the Adonis architecture, print its qubit connectivity and description

In [ ]:
adonis = Adonis()
print(adonis.__doc__)
print(adonis.metadata.gateset)
print(adonis.qubits)

# Creating a quantum circuit

Create a quantum circuit and insert native gates

In [ ]:
a, b, c = adonis.qubits[:3]
circuit = cirq.Circuit()
circuit.append(cirq.X(a))
circuit.append(cirq.PhasedXPowGate(phase_exponent=0.3, exponent=0.5)(c))
circuit.append(cirq.CZ(a, c))
circuit.append(cirq.YPowGate(exponent=1.1)(c))
print(circuit)

Insert non-native gates and decompose them into native ones by calling `decompose_circuit`

In [ ]:
circuit.append(cirq.ZZPowGate(exponent=0.2, global_shift=-0.5)(a, c))
circuit.append(cirq.HPowGate(exponent=-0.4)(a))
circuit = adonis.decompose_circuit(circuit)
print(circuit)

# Optimizing a quantum circuit

Use the `simplify_circuit` method to run a sequence of optimization passes on a circuit

In [ ]:
circuit = cirq.Circuit([
    cirq.H(a),
    cirq.CNOT(a, c),
    cirq.measure(a, c, key='result')])
print(circuit)

In [ ]:
circuit = adonis.decompose_circuit(circuit)
circuit = simplify_circuit(circuit)
print(circuit)

# Simulating a quantum circuit

Circuits that contain IQM-native gates can be simulated using the standard Cirq simulators

In [ ]:
sim = cirq.Simulator()
samples = sim.run(circuit, repetitions=100)

print('Samples:')
print(samples.histogram(key='result'))
print('\nState before the measurement:')
result = sim.simulate(circuit[:-1])
print(result)

Note that the above output vector represents the state before the measurement in the optimized circuit, not the original one, which would have the same phase for both terms. `simplify_circuit` has eliminated a `ZPowGate` which has no effect on the measurement.

---

# Creating a quantum circuit from an OpenQASM 2.0 program


In [ ]:
qasm_program = """
    OPENQASM 2.0;
    include "qelib1.inc";
    qreg q[3];
    creg meas[3];
    rx(1.7) q[1];
    h q[0];
    cx q[1], q[2];
"""
circuit = circuit_from_qasm(qasm_program)
print(circuit)

Decompose the circuit for the Adonis architecture

In [ ]:
decomposed = adonis.decompose_circuit(circuit)
print(decomposed)

Map the circuit qubits to device qubits manually

In [ ]:
qubit_mapping = {cirq.NamedQubit(k): v for k, v in {'q_0': a, 'q_1': b, 'q_2': c}.items()}
mapped = decomposed.transform_qubits(qubit_mapping)
print(mapped)

or automatically

In [ ]:
mapped, _, _ = adonis.route_circuit(decomposed)
print(mapped)

See the `examples` directory for more examples.

---

# The Apollo architecture

This section describes the same workflow using the 20-qubit Apollo architecture.

Construct an `IQMDevice` instance representing the Apollo architecture, print its qubit connectivity and description

In [ ]:
apollo = Apollo()
print(apollo.__doc__)
print(apollo.metadata.gateset)
print(apollo.qubits)

Create a circuit and insert native gates

In [ ]:
a, b, c, d, e, f = apollo.qubits[:6]
circuit = cirq.Circuit()
circuit.append(cirq.CZ(a, b))
circuit.append(cirq.CZ(a, c))
circuit.append(cirq.YPowGate(exponent=1.1)(d))
circuit.append(cirq.YPowGate(exponent=1.1)(e))
circuit.append(cirq.PhasedXPowGate(phase_exponent=0.3, exponent=0.5)(f))
print(circuit)

Insert non-native gates and decompose the circuit so that it contains native gates only

In [ ]:
circuit.append(cirq.ZZPowGate(exponent=0.2, global_shift=-0.5)(d, e))
circuit.append(cirq.HPowGate(exponent=-0.4)(f))
circuit = apollo.decompose_circuit(circuit)
print(circuit)

Use the `simplify_circuit` function to run a sequence of optimization passes on a circuit

In [ ]:
circuit = apollo.decompose_circuit(circuit)
circuit = simplify_circuit(circuit)
print(circuit)

In [ ]:
circuit.append(cirq.measure(a, f, key='result'))

sim = cirq.Simulator()
samples = sim.run(circuit, repetitions=100)

print('Samples:')
print(samples.histogram(key='result'))
print('\nState before the measurement:')
result = sim.simulate(circuit[:-1])
print(result)